<a href="https://colab.research.google.com/github/DavidMarshalov/DL/blob/main/BERT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import numpy as np
import evaluate

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch


In [ ]:
ds = load_dataset("MonoHime/ru_sentiment_dataset")

ds = ds.remove_columns(["Unnamed: 0"])
ds = ds.rename_column("sentiment", "labels")

np.unique(ds["train"]["labels"])

array([0, 1, 2])

In [ ]:
ds['train'][0]

{'text': '.с.,и спросил его:  о Посланник Аллаха!Ты порицаешь что-то из слушания?  Он ответил: я не порицаю ничего из него,но передай им,чтобы они открывали свои собрания Кораном и закрывали их Кораном ...........Это дошедшие до нас мнения и тот кто находится в поисках истины,по мере изучения этого вопроса будет сталкиваться с разногласиями и будет оставаться в растерянности или склонится к мнению одной из сторон по своему желанию.Но всего этого недостаточно,потому что он сам должен найти истину,подробно изучив вопросы запретного и разрешённого.|||||||||||||||||||||||||||||||||||||Обрати внимание:основатели всех четырёх мазхабов осуждали песни и поэззию.И этим часто грешат заблудшие суфии.Исключение делается для исламской тематики  Сахих БухариО проявлении радости во время праздника.498 (949). Сообщается, что ‘Аиша, да будет доволен ею Аллах, сказала: (Однажды) посланник Аллаха, да благословит его Аллах и приветствует, вошёл ко мне в то время, когда у меня находились две девушки, певши

In [ ]:
#данная модель имеет те же лейблы и аналогичным значением
model_name = "cointegrated/rubert-tiny2" #"blanchefort/rubert-base-cased-sentiment" #модель обучали на векипедии и новостях для задач связанных с тональностью
tok = AutoTokenizer.from_pretrained(model_name) #автоматически загружаем токенизатор под соответсвующую модель (он знает слвоарь данной модели с пец токены а атаже умеет паддить и обрезать до нужной длины)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
#проверка работоспособности
# clf = pipeline(
#     "text-classification",
#     model="blanchefort/rubert-base-cased-sentiment",
#     tokenizer="blanchefort/rubert-base-cased-sentiment"
# )

# clf("ненавижу но уважаю люблю и ценю ")

In [ ]:
#так как модель была обучена на одном типе данныъ то я могу просто дообучить модельку

In [ ]:
def tokenize(batch):
    return tok(
        batch["text"],                # токенизируем текстовые данные
        truncation=True,              # обрезаем слишком длинные тексты
        padding="max_length",         # паддинг до фиксированной длины
        max_length=128,               # фиксированная длина (можно менять)
    )

In [ ]:
tok_ds = ds.map(tokenize, batched=True)

Map:   0%|          | 0/189891 [00:00<?, ? examples/s]

Map:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [ ]:
print(tok_ds["train"].column_names)

['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
keep_cols = ["input_ids", "attention_mask", "labels"]
#закодированный текст маска внимания (какие токены настоящие (1) а какие паддинг) и лейбл

tok_ds = tok_ds.remove_columns(
    [c for c in tok_ds["train"].column_names if c not in keep_cols]
)

tok_ds.set_format(type="torch", columns=keep_cols)

In [ ]:
train_dataset = tok_ds["train"]
eval_dataset = tok_ds["validation"]

In [ ]:
tok_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 189891
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 21098
    })
})

In [ ]:
small_train = train_dataset.select(range(5000))
small_eval = eval_dataset.select(range(1500))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    #id2label=id2label,
    #label2id=label2id,
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#метрики

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1_macro = f1_metric.compute(predictions=preds,
                                 references=labels,
                                 average="macro")["f1"]
    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
    }

In [ ]:
#настройка аргументов обучения
training_args = TrainingArguments(
    output_dir="rubert-finetuned-fasttest",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=20,
    # save_steps=5000,
    # eval_steps=5000,
    # fp16=torch.cuda.is_available(),
    report_to=None,
)

#создание трейна

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    tokenizer=tok,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-2518122743.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.662400
40,0.630500
60,0.681200
80,0.607500
100,0.660200
120,0.635000
140,0.629500
160,0.633100
180,0.580500
200,0.592600


TrainOutput(global_step=314, training_loss=0.6229312617308015, metrics={'train_runtime': 559.7135, 'train_samples_per_second': 17.866, 'train_steps_per_second': 0.561, 'total_flos': 18437921280000.0, 'train_loss': 0.6229312617308015, 'epoch': 2.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.6685958504676819, 'eval_accuracy': 0.7013333333333334, 'eval_f1_macro': 0.6857243337973183, 'eval_runtime': 31.4099, 'eval_samples_per_second': 47.756, 'eval_steps_per_second': 0.764, 'epoch': 2.0}


In [ ]:
#модель с учетом обрезанного набора данных и настроенных параметров показала что она довольно стабильно различает 3 класса и не только за счет самого частого класса f1 примерно такой же


In [ ]:
#предобученная модель под задачу тональности

In [1]:
#данная модель имеет те же лейблы и аналогичным значением
#model_name = "blanchefort/rubert-base-cased-sentiment" #модель обучали на векипедии и новостях для задач связанных с тональностью
#tok = AutoTokenizer.from_pretrained(model_name) #автоматически загружаем токенизатор под соответсвующую модель (он знает слвоарь данной модели с пец токены а атаже умеет паддить и обрезать до нужной длины)

In [ ]:
#tok_ds = ds.map(tokenize, batched=True)

Map:   0%|          | 0/21098 [00:00<?, ? examples/s]

In [ ]:
#from transformers import AutoModelForSequenceClassification

#id2label = {
##    0: "NEUTRAL",
 #   1: "POSITIVE",
 ##   2: "NEGATIVE",
#}
#label2id = {v: k for k, v in id2label.items()}

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=3,
#     id2label=id2label,
#     label2id=label2id,
# )

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [2]:
# #настройка аргументов обучения
# training_args = TrainingArguments(
#     output_dir="blanchefort/rubert-base-cased-sentiment",
#     learning_rate=2e-5,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=64,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     logging_steps=20,
#     # save_steps=5000,
#     # eval_steps=5000,
#     # fp16=torch.cuda.is_available(),
#     report_to=None,
# )

# #создание трейна

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train,
#     eval_dataset=small_eval,
#     tokenizer=tok,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

In [ ]:
# metrics = trainer.evaluate()
# print(metrics)